### Generate a random set for quick testing

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import cellxgene_census
from sklearn.model_selection import train_test_split
from cellxgene_census.experimental import get_embedding, get_embedding_metadata, get_all_available_embeddings

# 1. Choose a Census version and organism
ORGANISM = "homo_sapiens"
MEASUREMENT = "RNA"
CENSUS_VERSION = "2025-01-30"

SAMPLE_SIZE = 5000

EMBEDDING_NAME = "geneformer"

In [12]:
with cellxgene_census.open_soma(census_version=CENSUS_VERSION) as census:
    adata = cellxgene_census.get_anndata(
        census,
        organism=ORGANISM,
        measurement_name=MEASUREMENT,
        obs_value_filter=f"soma_joinid < {SAMPLE_SIZE}",
        obs_embeddings=[EMBEDDING_NAME]
    )

/home/amoneim/.pyenv/versions/fmel/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/amoneim/.pyenv/versions/fmel/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [15]:
adata.obsm["geneformer"].shape

(5000, 512)

In [ ]:
# save adata
# adata.write("adata_sample.h5ad")